In [ ]:
from ultralytics import YOLO
import optuna
import yaml
import torch

In [ ]:
print(torch.cuda.is_available())
print(torch.version.cuda)
torch.cuda.empty_cache()

In [ ]:
# === Optuna ===

# === SETTINGS ===
MODEL_ARCH = "yolov8s.yaml"
DATASET_PATH = "dataset/dataset_day.yaml"
IMG_SIZE = 640
EPOCHS = 5
DEVICE = 0

def objective(trial):
    # HYPERPARAMETERS
    hyp = {
        'lr0': trial.suggest_float('lr0', 1e-4, 1e-1, log=True),
        'momentum': trial.suggest_float('momentum', 0.6, 0.98),
        'weight_decay': trial.suggest_float('weight_decay', 1e-5, 1e-2, log=True),
        'hsv_h': trial.suggest_float('hsv_h', 0.0, 0.1),
        'hsv_s': trial.suggest_float('hsv_s', 0.5, 1),
        'hsv_v': trial.suggest_float('hsv_v', 0.2, 1),
        'translate': trial.suggest_float('translate', 0.0, 0.2),
        'shear': trial.suggest_float('shear', 0.0, 2.0),
        'fliplr': trial.suggest_float('fliplr', 0.0, 0.5),
        'mosaic': trial.suggest_float('mosaic', 0.0, 1.0),
        'mixup': trial.suggest_float('mixup', 0.0, 0.5),
    }

    model = YOLO(MODEL_ARCH)

    model.train(
        data=DATASET_PATH,
        imgsz=IMG_SIZE,
        epochs=EPOCHS,
        batch=16,
        workers=4,
        device=DEVICE,
        optimizer='SGD',
        verbose=False,
        patience=2,
        lr0=hyp['lr0'],
        momentum=hyp['momentum'],
        weight_decay=hyp['weight_decay'],
        hsv_h=hyp['hsv_h'],
        hsv_s=hyp['hsv_s'],
        hsv_v=hyp['hsv_v'],
        translate=hyp['translate'],
        scale=1,
        shear=hyp['shear'],
        fliplr=hyp['fliplr'],
        mosaic=hyp['mosaic'],
        mixup=hyp['mixup'],
        project="optuna_yolo_tune",
        name=f"trial_{trial.number}",
    )

    # VALIDATE
    try:
        metrics = model.val()
        map50 = metrics.box.map50
    except Exception as e:
        print(f"Trial {trial.number} failed: {e}")
        return 0.0

    return map50

# === EXECUTE OPTUNA ===
study = optuna.create_study(
    direction="maximize",
    study_name="yolov8_tune",
    storage="sqlite:///optuna_yolov8.db",
    load_if_exists=True
)
study.optimize(objective, n_trials=10)

print("\n✅ Best hyperparameteres:")
for key, value in study.best_params.items():
    print(f"{key}: {value}")

# Zapisz do pliku YAML
with open("cfg/best_hyp.yaml", "w") as f:
    yaml.dump(study.best_params, f)

print("\n📄Saved best hyperparameters to best_hyp.yaml")